# Recurrent Neural Network

# ✅ 1) Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
import joblib
import os
from sklearn.metrics import mean_squared_error

# ✅ 2) Création du dossier output

In [2]:
os.makedirs('models', exist_ok=True)

# ✅ 3) Charger les datasets



In [4]:
dataset_train = pd.read_csv('Transformers_Training_Set.csv')
dataset_test = pd.read_csv('Transformers_Test_Set.csv')

# ✅ 4) Paramètres globaux

In [7]:
smell_names = dataset_train.columns[1:]  # sans 'Release version'
num_smells = len(smell_names)
timestep = 2

# ✅ 5) Boucle sur chaque smell

In [ ]:
print(smell_names)

for target_index, smell_name in enumerate(smell_names):
    print(f'=== Training for smell: {smell_name} ===')

    # Extraire colonnes [Release, smell]
    train_values = dataset_train.iloc[:, [target_index + 1]].values
    test_values = dataset_test.iloc[:, [target_index + 1]].values

    # Construire X_train, y_train une seule fois (avec scaling dans objective)
    dataset_total = np.concatenate((train_values, test_values), axis=0)

    # Scaler spécifique
    sc = MinMaxScaler(feature_range=(0, 1))
    train_scaled = sc.fit_transform(train_values)
    total_scaled = sc.transform(dataset_total)

    # Construire X_train, y_train
    X_train, y_train = [], []
    for i in range(timestep, len(train_scaled)):
        X_train.append(train_scaled[i - timestep:i])
        y_train.append(train_scaled[i])
    X_train, y_train = np.array(X_train), np.array(y_train)

    # Construire modèle
    regressor = Sequential()
    regressor.add(LSTM(units=64, return_sequences=True, input_shape=(timestep, 1)))
    regressor.add(Dropout(0.2))
    regressor.add(LSTM(units=64))
    regressor.add(Dropout(0.2))
    regressor.add(Dense(units=1))
    regressor.compile(optimizer='adam', loss='mean_squared_error')

    # Entraîner
    regressor.fit(X_train, y_train, epochs=100, batch_size=8, verbose=0)

    # Préparer X_test
    dataset_total = np.concatenate((train_values, test_values), axis=0)
    inputs_scaled = sc.transform(dataset_total)
    X_test = []
    for i in range(len(train_values), len(inputs_scaled)):
        X_test.append(inputs_scaled[i - timestep:i])
    X_test = np.array(X_test)

    # Prédire
    predicted_scaled = regressor.predict(X_test)
    predicted = sc.inverse_transform(predicted_scaled)

    # Courbe
    plt.figure(figsize=(10, 5))
    plt.plot(test_values, color='red', label='Valeurs réelles')
    plt.plot(predicted, color='blue', label='Prédictions')
    plt.title(f'Prédiction – Smell : {smell_name}')
    plt.xlabel('Release')
    plt.ylabel('Nombre de smells')
    plt.legend()
    plt.grid(True)
    plt.savefig(f'models/{smell_name}_prediction.png')
    plt.show()

    # Sauvegarder modèle et scaler
    regressor.save(f'models/{smell_name}_model.h5')
    joblib.dump(sc, f'models/{smell_name}_scaler.save')

    # Prédire la prochaine release
    last_inputs = inputs_scaled[-timestep:]
    last_inputs = np.expand_dims(last_inputs, axis=0)
    next_scaled = regressor.predict(last_inputs)
    next_value = sc.inverse_transform(next_scaled)
    print(f'👉 Prédiction pour la prochaine release ({smell_name}) : {next_value[0][0]:.2f}')

    # Evaluer sur test
    X_test = []
    for i in range(len(train_values), len(total_scaled)):
        X_test.append(total_scaled[i - timestep:i])
    X_test = np.array(X_test)
    predicted_scaled = regressor.predict(X_test, verbose=0)
    predicted = sc.inverse_transform(predicted_scaled)
    rmse = np.sqrt(mean_squared_error(test_values, predicted))

    print("RMSE :", rmse)



print("\n✅ Tous les modèles sont entraînés et sauvegardés dans le dossier 'models'.")
